In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import math

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/input.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            url = input_.loc[a, 'Url']

            dict_param = {}
            list_param = url.split('?')[1].split('&') if '?' in url else []
            for param in list_param:
                dict_param[param.split('=')[0]] = param.split('=')[1]
            
            dict_param.pop('start', '')
            dict_param.pop('num', '')
            
            url = url.split('?')[0] + '?'
            for key, value in dict_param.items():
                url += f'{key}={value}&'

            b = 0
            while True:
                b += 1
                try:
                    resp = requests.get(url[:-1],
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))
            
            # = = = = = = = = = = = = = = =

            count = int(html.xpath('//span[@id="lblResultCount"]/text()')[0].strip())

            # = = = = = = = = = = = = = = =

            if count <= 0:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                     'Url': url[:-1],
                                     'SKU Count': count,
                                     'Page': math.ceil(count / 100)}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        
            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/7.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/page_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：2] - 1246.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=License Lamp Lens
[剩余数量：1000] - [当前时间：22:17:28]

[状态：ok，尝试次数：2] - 1265.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Liftgate Latch Release Switch Seal
[剩余数量：999] - [当前时间：22:17:29]

[状态：ok，尝试次数：1] - 1262.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Liftgate Latch
[剩余数量：998] - [当前时间：22:17:29]

[状态：ok，尝试次数：3] - 1264.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Liftgate Latch Release Motor Gear Repair Kit
[剩余数量：997] - [当前时间：22:17:30]

[状态：ok，尝试次数：1] - 1268.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=Liftgate Object Sensor Retainer
[剩余数量：996] - [当前时间：22:17:30]

[状态：ok，尝试次数：2] - 1247.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=License Plate Bracket
[剩余数量：995] - [当前时间：22:17:30]

[状态：ok，尝试次数：33] - 1248.https://www.dormanproducts.com/gsearch.aspx?type=keyword&q=License Plate Bracket Bolt
[剩余数量：994] - [当前时间：22:17:30]

[状态：ok，尝试次数：13] - 1261.http